In [30]:
%pyspark
from pyspark import SparkFiles

# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/globalwarmingprj3/Countries_Lats.csv"

#  https://globalwarmingprj3.s3.amazonaws.com/Countries_Lats.csv
spark.sparkContext.addFile(url)

countries_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("Countries_Lats.csv"), inferSchema=True, sep=',')
countries_data_df.show(10)

UsageError: Cell magic `%%pyspark` not found.


In [3]:
%pyspark
url = "https://s3.amazonaws.com/globalwarmingprj3/GlobalLandTemperaturesByCountry-cleaned.csv"

#  https://globalwarmingprj3.s3.amazonaws.com/GlobalLandTemperaturesByCountry-cleaned.csv
spark.sparkContext.addFile(url)

land_temp_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("GlobalLandTemperaturesByCountry-cleaned.csv"), inferSchema=True, sep=',')
land_temp_data_df.show(10)

UsageError: Line magic function `%pyspark` not found.


In [4]:
%pyspark
url = "https://s3.amazonaws.com/globalwarmingprj3/greenhouse-gas-emissions-by-sector-cleaned.csv"

#  https://globalwarmingprj3.s3.amazonaws.com/greenhouse-gas-emissions-by-sector-cleaned.csv
spark.sparkContext.addFile(url)

green_house_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("greenhouse-gas-emissions-by-sector-cleaned.csv"), inferSchema=True, sep=',')
green_house_data_df.show(10)

UsageError: Line magic function `%pyspark` not found.


In [5]:
%pyspark
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://global-warming-rds.cvy0ocn3ij6n.us-east-2.rds.amazonaws.com:5432/global_warming_db"
config = {"user":"root", 
          "password": "globalwarming", 
          "driver":"org.postgresql.Driver"}


UsageError: Line magic function `%pyspark` not found.


In [6]:
%pyspark

from pyspark.sql.functions import col

# Create dataframe to match countries_lat_long table 

clean_countries_df = countries_data_df.select(col("code").alias("country_id"),col("latitude"),col("longitude"),col("country").alias("country_name"))


#clean_countries_df = countries_data_df.select(["country_id", "latitude", "longitude", "country_name"])
clean_countries_df.show()

UsageError: Line magic function `%pyspark` not found.


In [7]:
# Append DataFrame to active_user table in RDS
clean_countries_df.write.jdbc( jdbc_url,'country_lat_long',mode='append',properties=config);

NameError: name 'clean_countries_df' is not defined

In [8]:
%pyspark
#converting spark dataframe to pandas dataframe
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window  

land_temp_data_id_df = land_temp_data_df.withColumn('Id', row_number().over(Window.orderBy(monotonically_increasing_id())))
land_temp_data_id_df.show(10)


UsageError: Line magic function `%pyspark` not found.


In [9]:
%pyspark

#drop null values
drop_na_land_temp_df = land_temp_data_id_df.dropna()
drop_na_land_temp_df.count()

UsageError: Line magic function `%pyspark` not found.


In [10]:
%pyspark

from pyspark.sql.functions import col

# Create dataframe to match global_land_temps table 
clean_land_temp_df = drop_na_land_temp_df.select(col("Id").alias("Lt_id"),col("dt").alias("Lt_date"),col("AverageTemperature").alias("Avg_temp"),col("AverageTemperatureUncertainty").alias("Avg_temp_un"),col("code").alias("Country_id"))

clean_land_temp_df.show()

UsageError: Line magic function `%pyspark` not found.


In [11]:
%pyspark

#converting the date column from string to date format to insert into postgreSQL
#the date is converted to default spark format i.e "yyyy-mm-dd"
from pyspark.sql.functions import col, unix_timestamp, to_date

type_cnvrt_land_temp_df = clean_land_temp_df.withColumn('Lt_date', 
                   to_date(unix_timestamp(col('Lt_date'), 'dd/mm/yyyy').cast("timestamp")))

type_cnvrt_land_temp_df.show(10)

UsageError: Line magic function `%pyspark` not found.


In [12]:
%pyspark

# Append DataFrame to global_land_temps table in RDS
type_cnvrt_land_temp_df.write.jdbc(jdbc_url,'global_land_temps',mode='append',properties=config);

UsageError: Line magic function `%pyspark` not found.


In [14]:
%pyspark

from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window  

green_house_data_id_df = green_house_data_df.withColumn('Id', row_number().over(Window.orderBy(monotonically_increasing_id())))
green_house_data_id_df.count()

UsageError: Line magic function `%pyspark` not found.


In [15]:
%pyspark

#drop null values
drop_na_green_house_df = green_house_data_id_df.dropna()
drop_na_green_house_df.show(2)

UsageError: Line magic function `%pyspark` not found.


In [16]:
%pyspark

from pyspark.sql.functions import col

# Create dataframe to match green_house_gases table 
clean_green_house_df = drop_na_green_house_df.select(col("Id").alias("Gh_id"),col("Country").alias("Country"),col("Code").alias("Country_id"),col("Year").alias("Year"),col("Other sources (tonnes)").alias("Other_source"),col("Waste (tonnes)").alias("Waste"),col("Industry (tonnes)").alias("Industry"),col("Residential & commercial (tonnes)").alias("Residential"),col("Transport (tonnes)").alias("Transport"),col("Agriculture (tonnes)").alias("Agriculture"),col("Forestry (tonnes)").alias("Forestry"),col("Land use sources (tonnes)").alias("Land_use"),col("Energy (tonnes)").alias("Energy"))

#clean_land_temp_df.show()
clean_green_house_df.show(2)

UsageError: Line magic function `%pyspark` not found.


In [17]:
%pyspark

# Append DataFrame to green_house_gases table in RDS
clean_green_house_df.write.jdbc(jdbc_url,'green_house_gases',mode='append',properties=config);

UsageError: Line magic function `%pyspark` not found.


In [18]:
%pyspark
from pyspark.sql import SQLContext


green_house_from_db = sqlContext.read.jdbc(url=jdbc_url, table="green_house_gases", properties=config)
green_house_from_db.show()

UsageError: Line magic function `%pyspark` not found.


In [19]:
%pyspark
green_house_pandas_df = green_house_from_db.select("*").toPandas()
green_house_pandas_df.head(2)

UsageError: Line magic function `%pyspark` not found.


In [20]:
%pyspark
green_house_pandas_it = green_house_pandas_df.loc[green_house_pandas_df["country"]=="Italy",:]

green_house_pandas_x = green_house_pandas_it.loc[green_house_pandas_it["year"]>=1910,:]
green_house_pandas_x

green_house_pandas_y = green_house_pandas_it["waste"]
green_house_pandas_y

green_house_chart = green_house_pandas_y.plot(kind="bar")
plt.show()
#plt.bar(green_house_pandas_x, green_house_pandas_y, color='r', alpha=0.5, align="center")

UsageError: Line magic function `%pyspark` not found.


In [21]:
%pyspark
from pyspark.sql import SQLContext


global_land_from_db = sqlContext.read.jdbc(url=jdbc_url, table="global_land_temps", properties=config)
global_land_from_db.show()

UsageError: Line magic function `%pyspark` not found.


In [22]:
%pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

UsageError: Line magic function `%pyspark` not found.


In [23]:
%pyspark

global_land_pandas = global_land_from_db.select("*").toPandas()
#global_land_pandas

#change date column to date format to insert into postgres
global_land_pandas['lt_date'] = pd.to_datetime(global_land_pandas['lt_date'])
#global_land_pandas.info()

#Extracting the year from the date
global_land_pandas['year'] = pd.DatetimeIndex(global_land_pandas['lt_date']).year
global_land_pandas.head()

UsageError: Line magic function `%pyspark` not found.


In [24]:
%pyspark

years = np.unique(global_land_pandas['year'])

mean_temp_world = []
mean_temp_world_uncertainty = []

for year in years:
    mean_temp_world.append(global_land_pandas[global_land_pandas['year'] == year]['avg_temp'].mean())
    mean_temp_world_uncertainty.append(global_land_pandas[global_land_pandas['year'] == year]['avg_temp_un'].mean())

trace0 = go.Scatter(
    x = years, 
    y = np.array(mean_temp_world) + np.array(mean_temp_world_uncertainty),
    fill= None,
    mode='lines',
    name='Uncertainty top',
    line=dict(
        color='rgb(0, 255, 255)',
    )
)
trace1 = go.Scatter(
    x = years, 
    y = np.array(mean_temp_world) - np.array(mean_temp_world_uncertainty),
    fill='tonexty',
    mode='lines',
    name='Uncertainty bot',
    line=dict(
        color='rgb(0, 255, 255)',
    )
)

trace2 = go.Scatter(
    x = years, 
    y = mean_temp_world,
    name='Average Temperature',
    line=dict(
        color='rgb(199, 121, 093)',
    )
)
data = [trace0, trace1, trace2]

layout = go.Layout(
    xaxis=dict(title='year'),
    yaxis=dict(title='Average Temperature, °C'),
    title='Average land temperature in world',
    showlegend = False)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
#plt.show()



UsageError: Line magic function `%pyspark` not found.


In [25]:
%pyspark
x = years
y = mean_temp_world

#plot data
#plt.plot(x, y, marker="o", color="red")
plt.plot(x, y, color="red")
show(plt)

UsageError: Line magic function `%pyspark` not found.


In [26]:
%pyspark
#green_house_pandas_df.head(10)
mean_vals = []
mean_vals = green_house_pandas_df.mean()

mean_other = green_house_pandas_df['other_source'].mean()
mean_waste = green_house_pandas_df['waste'].mean()
mean_industry = green_house_pandas_df['industry'].mean()
mean_residenial = green_house_pandas_df['residential'].mean()
mean_transport = green_house_pandas_df['transport'].mean()
mean_agriculture = green_house_pandas_df['agriculture'].mean()
mean_forestry = green_house_pandas_df['forestry'].mean()
mean_land_use = green_house_pandas_df['land_use'].mean()
mean_energy = green_house_pandas_df['energy'].mean()

mean_vals

UsageError: Line magic function `%pyspark` not found.


In [27]:
%pyspark
pies = ["Other","Waste", "Industry", "Residential", "Transport", "Agriculture", "Forestry", "Land Use", "Energy"]
pie_votes = [mean_other,mean_waste,mean_industry,mean_residenial,mean_transport,mean_agriculture,mean_forestry,mean_land_use,mean_energy]
colors = ["yellow","green","lightblue","orange","red","purple","pink","yellowgreen","lightskyblue"]
explode = (0.1,0,0,0,0,0,0,0,0)

# Tell matplotlib to create a pie chart based upon the above data
plt.pie(pie_votes, explode=explode, labels=pies, colors=colors,autopct="%1.1f%%")
# Create axes which are equal so we have a perfect circle
plt.axis("equal")
# Save an image of our chart and print the final product to the screen
#plt.savefig("../Images/PyPies.png")
plt.show()

UsageError: Line magic function `%pyspark` not found.


In [28]:
%pyspark

green_house_pandas_us = green_house_pandas_df.loc[green_house_pandas_df["country_id"] == "US",:]

mean_other = green_house_pandas_us['other_source'].mean()
mean_waste = green_house_pandas_us['waste'].mean()
mean_industry = green_house_pandas_us['industry'].mean()
mean_residenial = green_house_pandas_us['residential'].mean()
mean_transport = green_house_pandas_us['transport'].mean()
mean_agriculture = green_house_pandas_us['agriculture'].mean()
mean_forestry = green_house_pandas_us['forestry'].mean()
mean_land_use = green_house_pandas_us['land_use'].mean()
mean_energy = green_house_pandas_us['energy'].mean()

pies = ["Other","Waste", "Industry", "Residential", "Transport", "Agriculture", "Forestry", "Land Use", "Energy"]
pie_votes = [mean_other,mean_waste,mean_industry,mean_residenial,mean_transport,mean_agriculture,mean_forestry,mean_land_use,mean_energy]
colors = ["yellow","green","lightblue","orange","red","purple","pink","yellowgreen","lightskyblue"]
explode = (0.1,0,0,0,0,0,0,0,0)

# Tell matplotlib to create a pie chart based upon the above data
plt.pie(pie_votes, explode=explode, labels=pies, colors=colors,autopct="%1.1f%%")
# Create axes which are equal so we have a perfect circle
plt.axis("equal")
# Save an image of our chart and print the final product to the screen
#plt.savefig("../Images/PyPies.png")
plt.show()

mean_other

UsageError: Line magic function `%pyspark` not found.


In [29]:
%pyspark
import plotly.io as pio



colors = ['blue', 'orange', 'green', 'red', 'purple']

opt = []
opts = []
for i in range(0, len(colors)):
    opt = dict(
        target = df['continent'][[i]].unique(), value = dict(marker = dict(color = colors[i]))
    )
    opts.append(opt)

data = [dict(
  type = 'scatter',
  mode = 'markers',
  x = df['lifeExp'],
  y = df['gdpPercap'],
  text = df['continent'],
  hoverinfo = 'text',
  opacity = 0.8,
  marker = dict(
      size = df['pop'],
      sizemode = 'area',
      sizeref = 200000
  ),
  transforms = [
      dict(
        type = 'filter',
        target = df['year'],
        orientation = '=',
        value = 2007
      ),
      dict(
        type = 'groupby',
        groups = df['continent'],
        styles = opts
    )]
)]

layout = dict(
    yaxis = dict(
        type = 'log'
    )
)

fig_dict = dict(data=data, layout=layout)
pio.show(fig_dict, validate=False)

SyntaxError: invalid syntax (<ipython-input-29-51381d149b74>, line 4)